Data Processing

Filter HTML files

In [6]:
import sqlite3
import pandas as pd

In [7]:
database = 'html.sqlite'
sql = """
SELECT filename, title, text
FROM html_attrs
"""

con = sqlite3.connect(database)
html_df = pd.read_sql_query(sql, con)
con.close()

html_df.dtypes

filename    object
title       object
text        object
dtype: object

Anfrage:
    Schlüsselwörter

In [9]:
word = "Schiedermeier"

[text for text in html_df["text"] if word in text][:5]

['       Allgemeine Studienberatung  Allgemeine Beratung erhalten Sie von der zentralen  Studienberatung. Die persönlichen Anforderungen, die für ein erfolgreiches Studium wichtig sind, haben wir zusammengestellt.        Studienfachberatung Informatik, Medieninformatik und Wirtschaftsinformatik  Wir wollen, dass Sie erfolgreich studieren! Wir beraten Sie vor Beginn und während des Studiums. Wir informieren Sie in regelmäßigen Veranstaltungen. Wir vermitteln Ansprechpartner wie Professoren, Kommissionen, Externe. Sie erreichen uns zu den ausgehängten Sprechzeiten bzw. über Telefon oder per E-Mail.            Bachelor - Studiengang Informatik - Studiengang Medieninformatik       Bachelor - Studiengang Informatik - Studiengang Medieninformatik         Name  Kontakt            Christian  Schiedermeier     Prof. Dr.-Ing.      telefon  +49 (0)911 5880 - 1179     email  christian.schiedermeier at th-nuernberg Punkt de     fax  +49 (0)911 5880 - 5666               Bachelor - Studiengang Wirtsc

Word embeddings für jedes Dokument erstellen.
BERT Embeddings
BERT (Google) trainiert auf 10.000+ Büchern
es gibt Modelle "base" und "large"
uncased heißt ohne klein - Großschreibung

Wir brauchen zur vorbereitung die zusätzlichen Token
[SEP] um das Ende eines Satzes zu markieren
[CLS] am Anfang des Texten
[PAD] zum auffüllen der Token 
Außerdem
TokenIDs
MaskIDs - zum filtern der [PAD]
Segment IDs um verschiedene Sätze zu unterscheiden
Posititional Embeddings


In [10]:
from transformers import BertModel, BertTokenizer
import torch

/Users/br/Projects/IT-Ptojekt-Chatbot/daibl/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [104]:
model = BertModel.from_pretrained('bert-base-uncased')

testSentence = "In der Bibliothek gibt es 40 Bücher zu Thema Animes"

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokens = tokenizer.tokenize(testSentence)

In [22]:
html_df["tokens"] = [tokenizer.tokenize(text) for text in html_df["text"]]

In [27]:
print(html_df["text"][2])
print(html_df["tokens"][2])

       Eine Pilotstudie. Oder: Wer kümmert sich um das Ehrenamt?  In der Praxis engagementfördernder Strukturen leisten „Verantwortliche für Engagement“ in Kommunen, Vereinen und Verbänden einen wichtigen Beitrag zum Gelingen des Engagements. Seit längerem ist dies u.a. ein Feld der Professionalisierung für Absolvent:innen des Studiengangs Soziale Arbeit. In der bisherigen Forschung wurde deren berufliches Setting jedoch kaum beleuchtet. Das Pilotprojekt wird hier erste valide Erkenntnisse bringen.    Laufzeit: 1. April 2022 bis 31. März 2023            Beteiligte         Projektleitung: Prof. Dr. Doris Rosenkranz (Fakultät SW) Projektmitarbeiter/in: Enya Buchner, M.A.           Zielsetzung         Verantwortliche für Engagement gibt es bundesweit in jedem Feld des Engagements. Doch bereits im „Wording“ für diese Funktionen zeigt sich eine große Heterogenität. Die These: Nach Organisation, Rahmenbedingungen oder Engagementfeld unterscheiden sich Bezeichnungen, berufliche Settings, Zuga

In [24]:
html_df["tokens"] = [["[SEP]" if token == "." else token for token in listl] for listl in html_df["tokens"]]

In [26]:
html_df["tokens"] = [["[CLS]"] + listl for listl in html_df["tokens"]]

In [30]:
html_df["token_ids"] = [tokenizer.convert_tokens_to_ids(token_list) for token_list in html_df["tokens"]]

In [35]:
html_df["token_ids"][2]

[101,
 27665,
 8221,
 8525,
 10265,
 102,
 27215,
 1024,
 2057,
 2099,
 13970,
 15810,
 2102,
 14387,
 2232,
 8529,
 8695,
 15501,
 7389,
 3286,
 2102,
 1029,
 1999,
 4315,
 10975,
 8528,
 2483,
 8147,
 3877,
 11795,
 4063,
 2358,
 6820,
 25509,
 5397,
 2078,
 26947,
 16173,
 1525,
 12297,
 3372,
 12155,
 5339,
 27412,
 6519,
 8147,
 1523,
 1999,
 12849,
 7382,
 9816,
 2078,
 1010,
 2310,
 2890,
 21820,
 6151,
 12034,
 5685,
 2368,
 27665,
 2078,
 15536,
 10143,
 29206,
 28236,
 29181,
 16950,
 2213,
 21500,
 15542,
 4078,
 20583,
 102,
 7367,
 4183,
 21395,
 28578,
 21541,
 8289,
 1057,
 102,
 1037,
 102,
 16417,
 10768,
 6392,
 4315,
 2658,
 17417,
 2121,
 5575,
 6519,
 14689,
 4747,
 15338,
 1024,
 7601,
 2368,
 4078,
 16054,
 9013,
 24930,
 2015,
 2061,
 12871,
 2571,
 12098,
 19205,
 2102,
 102,
 1999,
 4315,
 20377,
 5886,
 29206,
 2005,
 11624,
 5575,
 8814,
 25547,
 4315,
 2368,
 2022,
 6820,
 10258,
 17322,
 2015,
 4292,
 24401,
 11663,
 10556,
 2819,
 19337,
 13765,
 10143,
 

In [33]:
import numpy as np

ValueError: setting an array element with a sequence.

In [97]:

arr = np.array(html_df["token_ids"])
numeric_array = numpy_array.astype(np.float32)

tensor = torch.tensor(numeric_array)


In [ ]:

print(tokens)

['[CLS]', 'in', 'der', 'bi', '##bl', '##iot', '##he', '##k', 'gi', '##bt', 'es', '40', 'bu', '##cher', 'zu', 'them', '##a', 'anime', '##s', '[SEP]']


In [ ]:
#tokens = ['[CLS]'] + tokens + ['[SEP]']

In [101]:
attention_mask = [1 if token != "[PAD]" else 0  for token in tokens]

In [105]:
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print(token_ids)

[1999, 4315, 12170, 16558, 25185, 5369, 2243, 21025, 19279, 9686, 2871, 20934, 7474, 16950, 2068, 2050, 8750, 2015]


In [106]:
token_ids = torch.tensor(token_ids).unsqueeze(0)

attention_mask = torch.tensor(attention_mask).unsqueeze(0)

/var/folders/sx/swd0dqk97zvbv2327pzfkh800000gn/T/ipykernel_1456/1006243449.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(attention_mask).unsqueeze(0)


In [107]:
output = model(token_ids, attention_mask=attention_mask)

In [117]:
print(output[2])

IndexError: tuple index out of range

In [ ]:
output[0].shape

torch.Size([1, 20, 768])